In [65]:
import pandas as pd
import numpy as np

In [66]:
data = pd.read_csv("netflix_titles.csv")

In [3]:
data.shape

(8807, 12)

In [4]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [6]:
data.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [5]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'])

# Calculate cosine similarity between all pairs of movies
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
cosine_similarities.shape

(8807, 8807)

In [18]:
title_to_find = 'Ganglands'
movie_indices = data.index[data['title'] == title_to_find].tolist()

# Print the indices
print(f"Indices of movies with title '{title_to_find}': {movie_indices}")

Indices of movies with title 'Ganglands': [2]


In [35]:
def recommend_movie(title_to_find,num_rec):
    movie_indices = data.index[data['title'] == title_to_find].tolist()[0]
    print(movie_indices)
    similarities = cosine_similarities[movie_indices]
    sorted_indices = np.argsort(similarities)[::-1]
    top_n_movies_indices = sorted_indices[:num_rec+1]
    top_n_movie_titles = data.loc[top_n_movies_indices]['title'].tolist()
    return top_n_movie_titles[1:]

In [42]:
recommend_movie('Thor: Ragnarok',10)

8580


['Pandigai',
 'Lusers',
 'The Outsider',
 'Angel Beats!',
 'Inhuman Kiss',
 'Gour Hari Dastaan: The Freedom File',
 'Octonauts & the Great Barrier Reef',
 'IO',
 'Santa Clarita Diet',
 'Dukhtar']

In [126]:
data['description'] = data['description'].fillna('') + data['director'].fillna('') + data['listed_in'].fillna('') + data['type'].fillna('')

In [127]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk

In [128]:
tokenized_descriptions = [word_tokenize(desc.lower()) for desc in data['description']]

# Train Word2Vec model
embedding_size = 100  # You can adjust this based on your needs
model = Word2Vec(tokenized_descriptions, vector_size=embedding_size, window=5, min_count=1, sg=0)

# You can save the model for later use
model.save("movie_descriptions_word2vec.model")

In [129]:
embedding_vector = model.wv['action']
embedding_vector

array([-0.5184265 ,  1.4336095 , -1.1279651 ,  0.88217217, -1.2734706 ,
       -0.78887534, -0.6492172 ,  1.9589189 , -0.22012177, -3.234885  ,
       -0.6573134 , -0.31242514, -2.2324436 ,  0.05188342,  1.0118003 ,
        0.9074837 , -0.24146658,  0.51064444, -2.0512233 , -1.4412698 ,
        0.47179124,  0.8397431 ,  0.2969836 ,  0.4541243 ,  0.625643  ,
        1.3979965 , -1.7820641 , -0.7373029 , -0.7427942 ,  2.8741224 ,
        0.2065601 , -1.0646011 ,  2.603339  , -2.4967897 ,  0.10420928,
        2.4822884 ,  0.62088567,  2.4308279 ,  0.14125063,  1.7300371 ,
        0.82063913, -3.754287  , -2.241115  ,  1.4041905 ,  1.1542271 ,
       -2.9635465 , -0.24426544,  0.0709093 ,  1.9430627 ,  0.80214655,
        1.1739455 , -2.1500995 , -0.11427487, -0.25127465, -2.6248567 ,
        1.5922418 ,  1.3441266 ,  1.499648  , -0.6430934 ,  1.3876617 ,
        0.5905826 ,  1.0687064 ,  1.8153685 , -0.7929102 , -0.13924253,
        1.6574264 ,  0.9343099 ,  1.2155977 , -0.4150191 ,  1.18

In [130]:
def get_description_embedding(description):
    words = word_tokenize(description.lower())
    embedding = [model.wv[word] for word in words if word in model.wv]
    return sum(embedding) / len(embedding) if embedding else [0] * embedding_size

In [131]:
data['description_embedding'] = [get_description_embedding(desc) for desc in data['description']]

In [132]:
embedding_matrix = data['description_embedding']

In [133]:
description_embeddings = np.array(data['description_embedding'].to_list())

# Calculate cosine similarity between movie descriptions
cosine_similarities = cosine_similarity(description_embeddings, description_embeddings)

def recommend_movie(title_to_find, num_rec):
    movie_indices = data.index[data['title'] == title_to_find].tolist()[0]
    print(movie_indices)
    similarities = cosine_similarities[movie_indices]
    sorted_indices = np.argsort(similarities)[::-1]
    top_n_movies_indices = sorted_indices[:num_rec + 1]
    top_n_movie_titles = data.loc[top_n_movies_indices]['title'].tolist()
    
    # Get similarity scores for the recommended movies
    similarity_scores = [similarities[idx] for idx in top_n_movies_indices]
    
    # Create a list of tuples with movie titles and similarity scores
    recommended_movies_with_scores = list(zip(top_n_movie_titles[1:], similarity_scores[1:]))
    
    return recommended_movies_with_scores

In [134]:
recommended_movies = recommend_movie('Thor: Ragnarok',10)
print("Recommended Movies with Similarity Scores:")
for movie, score in recommended_movies:
    print(f"Movie: {movie}, Similarity Score: {score}")

8580
Recommended Movies with Similarity Scores:
Movie: The Matrix Reloaded, Similarity Score: 0.9980129599571228
Movie: Money Talks, Similarity Score: 0.9978792071342468
Movie: Spider-Man 3, Similarity Score: 0.9978455901145935
Movie: The Lord of the Rings: The Two Towers, Similarity Score: 0.9977781772613525
Movie: The Book of Eli, Similarity Score: 0.9975395798683167
Movie: In the Shadow of the Moon, Similarity Score: 0.997494101524353
Movie: Red Dawn, Similarity Score: 0.9973134398460388
Movie: Black Panther, Similarity Score: 0.9971238374710083
Movie: Seventh Son, Similarity Score: 0.9968461990356445
Movie: Ant-Man and the Wasp, Similarity Score: 0.9968146681785583


In [135]:
data.shape

(8807, 13)